# EIA Crude Oil Data Analysis using Machine Learning Technique

This research is mainly to utilize EIA api to import data, and apply machine learning techniques to predict Crude Oil Futures price movement, using all factors associated with EIA

Example of API data was given in the link below (Put your API key in the YOUR_API_KEY_HERE area in order to obtain data):
http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=PET.WCRFPUS2.W
http://api.eia.gov/series/?api_key=af034aec5e5242b2bc1ccaef0c93887c&series_id=PET.WCRFPUS2.W

All data are given in this link (hit the key button will show API link):
https://www.eia.gov/dnav/pet/pet_sum_sndw_dcus_nus_w.htm

In [300]:
import numpy as np
import pandas as pd
import requests

We will import data using requests packages, we will first try on 3 key data below:
Series ID = PET.WCRFPUS2.W #US Field Production of Crude Oil Weekly
Series ID = PET.WCRRIUS2.W #US Refiner Net Input of Crude Oil
Series ID = PET.WTTIMUS2.W #US Imports of Crude Oil and Petroleum Products, Weekly

In [301]:
api_key = "af034aec5e5242b2bc1ccaef0c93887c"
series_id = ['PET.WCRFPUS2.W', 'PET.WCRRIUS2.W', 'PET.WTTIMUS2.W']

df = pd.DataFrame() # Declare dataframe

for i in series_id:
    data = requests.get('http://api.eia.gov/series/?api_key=' + api_key + '&series_id=' + i).json()
    data = pd.DataFrame(data['series'])
    
    headers = ["time", i]
    tmp = data[['data']]
    tmp = tmp['data'][0]
    tmpdf = pd.DataFrame(tmp, columns = headers)
    df = pd.concat([df, tmpdf], axis = 1)


df = df.iloc[:, ~df.columns.duplicated()]
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values(by = ['time']) # Sort the dataframe ascending by time
df = df.dropna()

Below is how the dataframe looks like

In [302]:
df[0:5]

,time,PET.WCRFPUS2.W,PET.WCRRIUS2.W,PET.WTTIMUS2.W
1424,1991-02-08,7463.0,12973,6877.0
1423,1991-02-15,7427.0,12931,6573.0
1422,1991-02-22,7415.0,13107,6221.0
1421,1991-03-01,7404.0,13164,6188.0
1420,1991-03-08,7394.0,13082,7127.0


Obtaining Historical price data from Quandl api:
https://www.quandl.com/api/v3/datasets/CHRIS/CME_CL1

In [303]:
data = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/CME_CL1').json()

keys = list(data['dataset'].keys())
data = data['dataset']
price = data['data']
headers = data['column_names']
price_df = pd.DataFrame(price, columns = headers)
price_df = price_df.rename(columns = {'Date':'time'}) # Change Date column name into time to be consistent
price_df['time'] = pd.to_datetime(price_df['time'])
price_df = price_df.sort_values(by = ['time']) # Sort the dataframe ascending by time
price_df[0:5]

,time,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
8837,1983-03-30,29.01,29.56,29.01,29.40,NaN,29.40,949.0,470.0
8836,1983-03-31,29.40,29.60,29.25,29.29,NaN,29.29,521.0,523.0
8835,1983-04-04,29.30,29.70,29.29,29.44,NaN,29.44,156.0,583.0
8834,1983-04-05,29.50,29.80,29.50,29.71,NaN,29.71,175.0,623.0
8833,1983-04-06,29.90,29.92,29.65,29.90,NaN,29.90,392.0,640.0


Merge Factors Data and Historical pricing data

In [304]:
df = df.merge(price_df, left_on = 'time', right_on = 'time', how = 'left')
df[0:5]

,time,PET.WCRFPUS2.W,PET.WCRRIUS2.W,PET.WTTIMUS2.W,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
0,1991-02-08,7463.0,12973,6877.0,21.25,22.00,21.25,21.92,NaN,21.92,34407.0,62739.0
1,1991-02-15,7427.0,12931,6573.0,21.50,21.85,20.75,20.88,NaN,20.88,41124.0,38789.0
2,1991-02-22,7415.0,13107,6221.0,18.55,18.70,17.61,17.91,NaN,17.91,48592.0,61008.0
3,1991-03-01,7404.0,13164,6188.0,19.05,19.45,18.85,19.38,NaN,19.38,23272.0,54426.0
4,1991-03-08,7394.0,13082,7127.0,19.33,19.60,19.25,19.31,NaN,19.31,29235.0,55236.0
